In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import os
import numpy as np

Get the Data

In [13]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_values_file = 'nifty50_index_prices_2015_2025.csv'
index_cons_file = 'nifty50_cons.csv'

In [14]:
index_values = pd.read_csv(base_dir + index_values_file, usecols=['Date', 'Adj Close'], parse_dates=['Date'])
index_values['Index_daily_return'] = index_values['Adj Close'].pct_change()
index_values.dropna(inplace=True)
index_values

,Date,Adj Close,Index_daily_return
1,2015-01-05,8378.400391,-0.002031
2,2015-01-06,8127.350098,-0.029964
3,2015-01-07,8102.100098,-0.003107
4,2015-01-08,8234.599609,0.016354
5,2015-01-09,8284.500000,0.006060
...,...,...,...
2702,2025-12-23,26177.150391,0.000181
2703,2025-12-24,26142.099609,-0.001339
2704,2025-12-26,26042.300781,-0.003818
2705,2025-12-29,25942.099609,-0.003848


In [15]:
index_cons = pd.read_csv(base_dir + index_cons_file, parse_dates=['Date'], usecols=['Date', 'Security Symbol', 'Free Float Market Capitalisation (Rs. Crores)'])
index_cons.insert(1, 'Index_name', 'Nifty 50')
index_cons.rename(columns={'Free Float Market Capitalisation (Rs. Crores)': 'ff_mcap'}, inplace=True)
index_cons

,Date,Index_name,Security Symbol,ff_mcap
0,2016-03-31,Nifty 50,ACC,13003.10
1,2016-03-31,Nifty 50,INFY,243457.99
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08
3,2016-03-31,Nifty 50,LT,99713.71
4,2016-03-31,Nifty 50,LUPIN,35321.94
...,...,...,...,...
997,2025-09-30,Nifty 50,HINDALCO,NaN
998,2025-09-30,Nifty 50,HINDUNILVR,NaN
999,2025-09-30,Nifty 50,ICICIBANK,NaN
1000,2025-09-30,Nifty 50,COALINDIA,NaN


manually remove HDFC, ZOMATO as they have data under different ticker name

In [16]:
index_cons = index_cons[~index_cons['Security Symbol'].isin(['HDFC', 'ZOMATO'])]
index_cons

,Date,Index_name,Security Symbol,ff_mcap
0,2016-03-31,Nifty 50,ACC,13003.10
1,2016-03-31,Nifty 50,INFY,243457.99
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08
3,2016-03-31,Nifty 50,LT,99713.71
4,2016-03-31,Nifty 50,LUPIN,35321.94
...,...,...,...,...
997,2025-09-30,Nifty 50,HINDALCO,NaN
998,2025-09-30,Nifty 50,HINDUNILVR,NaN
999,2025-09-30,Nifty 50,ICICIBANK,NaN
1000,2025-09-30,Nifty 50,COALINDIA,NaN


In [17]:
index_cons['Security Symbol'] = index_cons['Security Symbol'].replace('TATAMOTORS', 'TMCV')
index_cons

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_10122/1888627043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Security Symbol'] = index_cons['Security Symbol'].replace('TATAMOTORS', 'TMCV')


,Date,Index_name,Security Symbol,ff_mcap
0,2016-03-31,Nifty 50,ACC,13003.10
1,2016-03-31,Nifty 50,INFY,243457.99
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08
3,2016-03-31,Nifty 50,LT,99713.71
4,2016-03-31,Nifty 50,LUPIN,35321.94
...,...,...,...,...
997,2025-09-30,Nifty 50,HINDALCO,NaN
998,2025-09-30,Nifty 50,HINDUNILVR,NaN
999,2025-09-30,Nifty 50,ICICIBANK,NaN
1000,2025-09-30,Nifty 50,COALINDIA,NaN


In [18]:
price_data = pd.read_csv(base_dir + 'all_price_data.csv', parse_dates=['Date'])
price_data

,Symbol,Date,Close,Volume
0,BHARTIARTL,2015-01-01,308.073578,2460188
1,BHARTIARTL,2015-01-02,309.898743,2795887
2,BHARTIARTL,2015-01-05,303.022461,1605267
3,BHARTIARTL,2015-01-06,300.645477,3916948
4,BHARTIARTL,2015-01-07,301.367035,4899318
...,...,...,...,...
188365,M&M,2025-12-23,3625.100098,1413628
188366,M&M,2025-12-24,3636.699951,842897
188367,M&M,2025-12-26,3623.100098,829910
188368,M&M,2025-12-29,3592.100098,1034692


In [19]:
price_data['daily_return'] = price_data.groupby('Symbol')['Close'].pct_change()
price_data.dropna(inplace=True)
price_data

,Symbol,Date,Close,Volume,daily_return
1,BHARTIARTL,2015-01-02,309.898743,2795887,0.005924
2,BHARTIARTL,2015-01-05,303.022461,1605267,-0.022189
3,BHARTIARTL,2015-01-06,300.645477,3916948,-0.007844
4,BHARTIARTL,2015-01-07,301.367035,4899318,0.002400
5,BHARTIARTL,2015-01-08,306.587952,7096620,0.017324
...,...,...,...,...,...
188365,M&M,2025-12-23,3625.100098,1413628,0.002323
188366,M&M,2025-12-24,3636.699951,842897,0.003200
188367,M&M,2025-12-26,3623.100098,829910,-0.003740
188368,M&M,2025-12-29,3592.100098,1034692,-0.008556


Calculate Beta and volatitlty of each constituent on index cons

In [20]:
def one_year_beta(symbol, value_date):
    start_date = pd.to_datetime(value_date) - pd.DateOffset(years=1)
    end_date = pd.to_datetime(value_date)
    
    stock_data = price_data[(price_data['Symbol'] == symbol) & 
                            (price_data['Date'] >= start_date) & 
                            (price_data['Date'] <= end_date)]
    index_data = index_values[(index_values['Date'] >= start_date) & 
                             (index_values['Date'] <= end_date)]
    
    if stock_data.empty or index_data.empty:
        return np.nan  # Return NaN if data is insufficient
    
    merged_data = pd.merge(stock_data, index_data, left_on='Date', right_on='Date', suffixes=('_stock', '_index'))
    if merged_data.empty:
        return np.nan  # Return NaN if no merged data
    
    X = merged_data['Index_daily_return']
    y = merged_data['daily_return']
    
    if len(X) < 2:  # Ensure enough data points for regression
        return np.nan
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.params['Index_daily_return']

# Apply the function with error handling
index_cons['Beta'] = index_cons.apply(
    lambda row: one_year_beta(row['Security Symbol'], row['Date']), axis=1
)

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_10122/1148718232.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Beta'] = index_cons.apply(


In [21]:
def one_year_volatility(symbol, value_date):
    start_date = pd.to_datetime(value_date) - pd.DateOffset(years=1)
    end_date = pd.to_datetime(value_date)

    stock_data = price_data[(price_data['Symbol'] == symbol) & 
                            (price_data['Date'] >= start_date) & 
                            (price_data['Date'] <= end_date)]

    if stock_data.empty:
        return np.nan  # Return NaN if data is insufficient

    # Calculate daily returns
    daily_returns = stock_data['daily_return']

    if len(daily_returns) < 2:  # Ensure enough data points for calculation
        return np.nan

    # Calculate annualized volatility
    volatility = np.std(daily_returns) * np.sqrt(252)  # Assuming 252 trading days in a year
    return volatility
# Apply the function with error handling
index_cons['Volatility'] = index_cons.apply(
    lambda row: one_year_volatility(row['Security Symbol'], row['Date']), axis=1) 

/var/folders/7g/dzc0yw6s4h19dccysb1yd3rc0000gn/T/ipykernel_10122/84915883.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_cons['Volatility'] = index_cons.apply(


Delete the tickers with missing beta or volatilty due to insufficient data

In [22]:
index_cons = index_cons.dropna(subset=['Beta', 'Volatility'])
index_cons

,Date,Index_name,Security Symbol,ff_mcap,Beta,Volatility
0,2016-03-31,Nifty 50,ACC,13003.10,0.840607,0.238163
1,2016-03-31,Nifty 50,INFY,243457.99,0.818393,0.274475
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08,1.094524,0.274023
3,2016-03-31,Nifty 50,LT,99713.71,1.193147,0.285038
4,2016-03-31,Nifty 50,LUPIN,35321.94,0.734866,0.344207
...,...,...,...,...,...,...
997,2025-09-30,Nifty 50,HINDALCO,NaN,1.216176,0.277803
998,2025-09-30,Nifty 50,HINDUNILVR,NaN,0.441855,0.195020
999,2025-09-30,Nifty 50,ICICIBANK,NaN,0.880423,0.173273
1000,2025-09-30,Nifty 50,COALINDIA,NaN,0.933043,0.213540


In [23]:
index_cons.to_csv(os.path.join(base_dir, 'nifty50_cons_with_risk_metrics.csv'), index=False)